
---

## **Atividade 1 — Classificação Supervisionada**

Nesta atividade, você deve **comparar quatro diferentes paradigmas de aprendizado de máquina**: probabilísticos, simbólicos, conexionistas e estatísticos.

> **Importante:** Durante a **etapa de inferência**, o modelo **só poderá utilizar o conteúdo textual** (`content`) do post para prever o engajamento. Informações como `reactions` e `comments` devem ser utilizadas apenas na etapa de treinamento, se desejar, mas não podem ser usadas como entrada no momento da predição de novos posts (eles não estarão disponíveis em inferência).

Para avaliação, será disponibilizado um arquivo de teste contendo **novas postagens sem o rótulo de engajamento**, e cada aluno deverá submeter suas **previsões finais de engajamento** para essas novas entradas.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
#from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB


In [ ]:
df = pd.read_pickle('../data/df_social_data_train.pkl')
df

In [ ]:
# pre-processar e extrair caracteristicas dos dados para construir a tabela atributo-valor
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
df = df.dropna()

In [ ]:
df['features'] = list(model.encode(df['content'].tolist(), show_progress_bar=True))

df

In [ ]:
df['engagement'] = df['engagement'].map({'low': 0, 'high': 1})

In [ ]:
labels = df['engagement'].to_numpy()
data = np.array(df['features'].tolist())

In [ ]:
kf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)

def applyKfold(data, label, classifier):

    accVet = []
    f1Vet = []
    f1MacroVet = []

    for train_index, test_index in kf.split(data, label):
        # particição dos dados em conjunto de treino / teste
        data_train, data_test = data[train_index], data[test_index]
        label_train, label_test = label[train_index], label[test_index]


        # classificador
        clf = classifier
        clf.fit(data_train, label_train)

        # predição
        y_pred = clf.predict(data_test)

        # avaliação
        acc = accuracy_score(label_test, y_pred)
        f1 = f1_score(label_test, y_pred)
        f1_macro = f1_score(label_test, y_pred, average='macro')

        accVet.append(acc)
        f1Vet.append(f1)
        f1MacroVet.append(f1_macro)


    return (accVet, f1Vet, f1MacroVet)






---
## Abordagem **Probabilística** - Naive Bayes

In [ ]:
acc, f1, f1Macro = applyKfold(data, labels, GaussianNB())
print("Accuracy Score: ", round(np.mean(acc), 4))
print("F1 Score: ", round(np.mean(f1), 4))
print("F1 Macro Score: ", round(np.mean(f1Macro), 4))


---
## Abordagem **Simbólica** - Decision Tree

In [ ]:
acc, f1, f1Macro = applyKfold(data, labels, DecisionTreeClassifier(criterion='entropy'))
print("Accuracy Score: ", round(np.mean(acc), 4))
print("F1 Score: ", round(np.mean(f1), 4))
print("F1 Macro Score: ", round(np.mean(f1Macro), 4))


---
## Abordagem **Conexionista** - Multilayer Perceptron

In [ ]:
# MLP (com regularização e early stopping)
value_1 = 32
value_2 = 16
value_3 = 2

mlp = Sequential()
mlp.add(Dense(value_1, input_shape=(data.shape[1],), activation='tanh'))
mlp.add(Dropout(0.1))
mlp.add(Dense(value_2, input_shape=(data.shape[1],), activation='tanh'))
mlp.add(Dropout(0.1))
mlp.add(Dense(1, input_shape=(data.shape[1],), activation='sigmoid'))

mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
mlp.summary()

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

history = mlp.fit(
    X_train,
    y_train,
    epochs=1000,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
# Plotando curva de aprendizado
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()
plt.title("Curva de Aprendizado")
plt.show()

In [ ]:
# Avaliar no conjunto de teste
test_loss, test_acc = mlp.evaluate(X_test, y_test, verbose=1)
print(f"[MLP] Acurácia no teste final: {test_acc:.4f}, Loss: {test_loss:.4f}")


---
## Abordagem **Estatística** - SVC

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# classificador
clf_svc = SVC(kernel="rbf", C=10, gamma='scale', max_iter=10000)
clf_svc.fit(X_train_scaled, y_train)

# predição
y_pred = clf_svc.predict(X_test_scaled)

# avaliação
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='macro')

print("Accuracy Score:", round(acc, 4))
print("F1 Score:", round(f1, 4))
print("F1 Macro Score:", round(f1_macro, 4))